## Connecting to the jlab portal

1. Please connect to the [jlab portal](https://jlab2.lxp-prod.cloud.lxp.lu) first.

2. Start a server. Don't forget to select FPGA instead of CPU (see below):


<img src="https://github.com/LuxProvide/QuantumFPGA/blob/main/docs/images/jlab.png?raw=true" width="500">
  
3. Clone the training repository: `git clone https://github.com/LuxProvide/QuantumFPGA`

4. Finally, open the notebook `qcfpa.ipynb` in JupyterLab.


## Setting the IPython kernel

We need now to create a dedicated IPython kernel to be able to run efficiently on efficiently on FPGA node

- Kernels are by default located in this folder `$HOME/.local/share/jupyter/kernels`

- Execute the following cell to create a custom kernel named **QCFPGA**

In [3]:
%%bash
KERNEL="$HOME/.local/share/jupyter/kernels/qcfpga"
mkdir -p $KERNEL
PRELOAD="$KERNEL/start.sh"
JSON="$KERNEL/kernel.json"
cat << 'EOF' > $JSON
{
 "argv": [
  "{resource_dir}/start.sh",
  "python",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "QCFPGA",
 "language": "python",
 "metadata": {
  "debugger": true
 }
}
EOF

cat << 'EOF' > $PRELOAD
#!/bin/bash
module load QCFPGA
module load jemalloc
export JEMALLOC_PRELOAD=$(jemalloc-config --libdir)/libjemalloc.so.$(jemalloc-config --revision)
export LD_PRELOAD=${JEMALLOC_PRELOAD}
export PYOPENCL_COMPILER_OUTPUT=1
exec "$@"
EOF

chmod u+x $PRELOAD 

- Execute the following cell to reload everything

In [3]:
%load_ext autoreload
%autoreload 2

- Now change the current kernel by the new one: **`Kernel --> Change Kernels --> QCFPGA`**
- If you see a white circle, the kernel is ready
  
<img src="https://github.com/LuxProvide/QuantumFPGA/blob/main/docs/images/kernel_ready.png?raw=true" width="500">

## The QCFPGA library

**QCFPGA** is a software library which is a fork of the public [QCGPU software](https://github.com/libtangle/qcgpu) that was designed to perform quantum computing simulations on graphics processing units (GPUs) using [PyOpenCL](./pyopencl_ifpgasdk.md). The main idea behind QCFPGA is to utilize the parallel processing capabilities of modern FPGAs to speed up quantum simulations, which are computationally intensive tasks that can benefit greatly from the pipeline parallelism offered by modern FPGAs. 

The library provides a high-level interface for defining quantum states, applying gates, and performing measurements, much like other quantum computing frameworks. Nonetheless, the library is far from being complete as the Qiskit (IBM) or Cirq (Google).

QFPGA was adapted from QCGPU as a **proof of concept** with the intent to make quantum computing simulations more accessible and faster, leveraging the powerful computational capabilities of FPGAs to handle state vector manipulations typical in quantum computing. 


<div class="alert alert-block alert-info">
⚠️  QCFPGA is a Work In Progress and may be subject to changes in the near future. Our main goal is to take advantage of kernel optimization on FPGAs and develop a multi-node version.For any problem, please contact the support team using our <a href="https://servicedesk.lxp.lu/"> servicedesk portal</a>
</div>

In [4]:
# Import QCFPGA
import qcfpga
import numpy as np

# Create a new quantum register with 1 qubits
register = qcfpga.State(1)
# Let's try the Hadamard gate
register.h(0)
np.round(register.probabilities(),2)

array([0.5, 0.5], dtype=float32)

### Built-In Gates

In Quantum Computing, gates are used to manipulate quantum registers and
to implement quantum algorithms.

There are a number of gates built into **QCGPU** and **QCFPGA**. They can all be applied the same way:

In [11]:
register = qcfpga.State(2)

register.h(0) # Applies the Hadamard  gate to the first qubit.
register.x(1) # Applies a pauli-x  gate to the second qubit.
np.round(register.probabilities(),2)

array([0. , 0. , 0.5, 0.5], dtype=float32)

These are the gates that can be applied to a register:

- The Hadamard gate: **h** - `state.h(0)`

- The S gate: **s** - `state.s(0)`

- The T gate: **t** - `state.t(0)`

- The Pauli-X / NOT gate: **x** - `state.x(0)`

- The Pauli-Y gate: **y** - `state.y(0)`

- The Pauli-Z gate: **z** - `state.z(0)`

- The CNOT gate: **cx** -`state.cx(0, 1) # CNOT with control = 0, target = 1`

- The SWAP gate: **swap** -`state.swap(0,1) # Swaps the 0th and 1st qubit`

- The Toffoli gate: **toffoli** -`state.toffoli(0, 1, 2) # Toffoli with controls = (0, 1), target = 2`


In [12]:
''' 
For example, you can also use any of the gates as controlled gates. 
Controlled gates can be also used to entangle state
'''
x = qcfpga.gate.x()
h = qcfpga.gate.h()

register = qcfpga.State(5)
register.apply_gate(h,0)
register.apply_controlled_gate(x, 0, 1)
np.round(register.probabilities(),2)

array([0.5, 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. ], dtype=float32)

<img src="https://github.com/LuxProvide/QuantumFPGA/blob/main/docs/images/quirks_cnot.png?raw=true" width="500">

### Applying a gate to all qubits in parallel

In [16]:
'''
It is also trivial to apply a gate to all qubit of a register
'''
h = qcfpga.gate.h()

register = qcfpga.State(3)
register.apply_all(h)
np.round(register.probabilities(),3)


array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125],
      dtype=float32)

<img src="https://github.com/LuxProvide/QuantumFPGA/blob/main/docs/images/quirks_all.png?raw=true" width="800" >

### Define your own gate

- Custom gates in QCFPGA use the `qcfpga.Gate` class.

- Only single gate qubits can be defined

<div class="alert alert-block alert-info">
⚠️  The input to the `Gate` constructor is checked to be a 2x2 unitary matrix.
</div>


In [17]:
gate_matrix = np.array([
    [1, 0],
    [0, np.exp(1j * np.pi / 4)]
])

gate = qcfpga.Gate(gate_matrix)
register = qcfpga.State(3)
register.apply_all(gate)
np.round(register.probabilities(),3)


array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)